In [1]:
from IPython.display import Audio
from scipy.io.wavfile import write as write_wav

from bark.api import generate_audio
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

In [2]:
semantic_path = "semantic_output/pytorch_model.bin" # set to None if you don't want to use finetuned semantic
coarse_path = "coarse_output/pytorch_model.bin" # set to None if you don't want to use finetuned coarse
fine_path = "fine_output/pytorch_model.bin" # set to None if you don't want to use finetuned fine
use_rvc = True # Set to False to use bark without RVC
rvc_name = 'howard-test'
rvc_path = f"Retrieval-based-Voice-Conversion-WebUI/assets/weights/{rvc_name}.pth"
index_path = f"Retrieval-based-Voice-Conversion-WebUI/logs/{rvc_name}/added_IVF256_Flat_nprobe_1_{rvc_name}_v2.index"
device="cuda:0"
is_half=True

In [9]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    text_model_path=semantic_path,
    coarse_use_gpu=True,
    coarse_use_small=False,
    coarse_model_path=coarse_path,
    fine_use_gpu=True,
    fine_use_small=False,
    fine_model_path=fine_path,
    codec_use_gpu=True,
    force_reload=False,
    path="models"
)

if use_rvc:
    from rvc_infer import get_vc, vc_single
    vc = get_vc(rvc_path, device, is_half)
    vc.get_vc("howard-test.pth")

loading pth Retrieval-based-Voice-Conversion-WebUI/assets/weights/howard-test.pth
<All keys matched successfully>


2023-09-20 20:30:39 | INFO | modules | Get sid: howard-test.pth
2023-09-20 20:30:39 | INFO | modules | Loading: ./Retrieval-based-Voice-Conversion-WebUI/assets/weights/howard-test.pth


@@@@ instantiating VC, tgt_sr 40000, config <rvc_infer.Config object at 0x00000208F7339F50>


2023-09-20 20:30:39 | INFO | modules | Select index: 0


In [10]:
# simple generation
text_prompt = "Hello, my name is Serpy. And, uh — and I like pizza. [laughs]"
filepath = "output/audio.wav"

voice_name = "howard-clone-from-60sec-sample" # use your custom voice name here if you have on
#audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)
#write_wav(filepath, SAMPLE_RATE, audio_array)

In [11]:
print(vc)

if use_rvc:
    index_rate = 0.75
    f0up_key = -6
    filter_radius = 3
    rms_mix_rate = 0.25
    protect = 0.33
    resample_sr = SAMPLE_RATE
    f0method = "harvest" #harvest or pm
    try:
        audio_array = vc.vc_single(0,filepath,f0up_key,None,f0method,index_path, None, index_rate, filter_radius=filter_radius, resample_sr=resample_sr, rms_mix_rate=rms_mix_rate, protect=protect)
    except:
        audio_array = vc.vc_single(0,filepath,f0up_key,None,'pm',index_path, None, index_rate, filter_radius=filter_radius, resample_sr=resample_sr, rms_mix_rate=rms_mix_rate, protect=protect)
    print(f"@@@ len out: {len(audio_array)}")
    write_wav("output/vc-out.wav", SAMPLE_RATE, audio_array)

Audio(audio_array, rate=SAMPLE_RATE)

C:\Users\guoho\.virtualenvs\bark-with-voice-clone-KGiCdjZF\Lib\site-packages\hydra\experimental\initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
C:\Users\guoho\.virtualenvs\bark-with-voice-clone-KGiCdjZF\Lib\site-packages\hydra\experimental\initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
C:\Users\guoho\.virtualenvs\bark-with-voice-clone-KGiCdjZF\Lib\site-packages\hydra\experimental\compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
C:\Users\guoho\.virtualenvs\bark-with-voice-clone-KGiCdjZF\Lib\site-packages\hydra\core\default_element.py:124: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See htt

@@@ len out: 2


AttributeError: 'tuple' object has no attribute 'dtype'

In [ ]:
# generation with more control
text_prompt = "Hello, my name is Serpy. And, uh — and I like pizza. [laughs]"
voice_name = "speaker_0" # use your custom voice name here if you have on

filepath = "output/audio.wav"

x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)
write_wav(filepath, SAMPLE_RATE, audio_array)

if use_rvc:
    index_rate = 0.75
    f0up_key = -6
    filter_radius = 3
    rms_mix_rate = 0.25
    protect = 0.33
    resample_sr = SAMPLE_RATE
    f0method = "harvest" #harvest or pm
    try:
        audio_array = vc_single(0,filepath,f0up_key,None,f0method,index_path,index_rate, filter_radius=filter_radius, resample_sr=resample_sr, rms_mix_rate=rms_mix_rate, protect=protect)
    except:
        audio_array = vc_single(0,filepath,f0up_key,None,'pm',index_path,index_rate, filter_radius=filter_radius, resample_sr=resample_sr, rms_mix_rate=rms_mix_rate, protect=protect)
    write_wav(filepath, SAMPLE_RATE, audio_array)

Audio(audio_array, rate=SAMPLE_RATE)